In [1]:
import pandas as pd








In [2]:
def load_kddi_data(data_path):
    col = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment",
           "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted",
           "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
           "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
           "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
           "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
           "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
           "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"]
    dataset = pd.read_csv(data_path, names=col)
    
    # 正常なデータを0, 悪意のあるデータを1とする
    labels = dataset["label"]
    labels = labels.replace({"^.*normal.*":0,"^(?!normal).*$":1}, regex=True)

    # 文字列とラベルを取り除く
    drop_columns = ["protocol_type", "service", "flag", "label"]
    return dataset.drop(drop_columns, axis=1), labels

In [3]:
def load_converted_data(data_path):
    col = ["num_conn", "startTimet", "orig_pt", "resp_pt", "orig_ht", "resp_ht",
           "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment",
           "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted",
           "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
           "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
           "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
           "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
           "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
           "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate"]
    dataset = pd.read_csv(data_path, names=col)

    # 余分な列と文字列を取り除く
    drop_columns = ["num_conn", "startTimet", "orig_pt", "resp_pt", "orig_ht", "resp_ht",
                    "protocol_type", "service", "flag"]
    return dataset.drop(drop_columns, axis=1)

In [4]:
#X_kddi, y_kddi = load_kddi_data('./dataset/kddcup99/kddcup.data_10_percent')
X_kddi, y_kddi = load_kddi_data('./dataset/kddcup99/kddcup.data')
#X_kddi

In [5]:
load_converted_path = './dataset/trafAld.list'
with open(load_converted_path, mode='r') as f:
    s = f.read().replace(' ', ',')
with open(load_converted_path, mode='w') as f:
    f.write(s)
    
X_conv = load_converted_data(load_converted_path)
#X_conv

In [6]:
import sklearn
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

#pred = KMeans(n_clusters=2, init='random', n_init=10, n_jobs=2).fit_predict(X_kddi)
#acc = accuracy_score(y_kddi, pred)
#print ("Accuracy: {:.3}%".format(acc*100))

Accuracy: 19.7%


In [7]:
import sklearn
import numpy as np
import pandas as pd
import discriminator
from sklearn.datasets import load_digits
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))  
#X = scaler.fit_transform(X_kddi)
#X = pd.DataFrame.from_records(X_kddi)
X = np.array(X_kddi.values)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = np.array(y_kddi.tolist())
#X_conv_test = scaler.fit_transform(X_conv)
#X_conv_test = pd.DataFrame.from_records(X_conv_test)
X_conv_test = np.array(X_conv.values)
X_conv_test = np.reshape(X_conv_test, (X_conv_test.shape[0], X_conv_test.shape[1], 1))

In [9]:
import discriminator

from keras.utils import np_utils

In [10]:
my_tsize = .4 # 40%
my_seed = 123
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=my_tsize, random_state=my_seed)
SAMPLE_SIZE=500
if SAMPLE_SIZE:
    X_train = X_train[:SAMPLE_SIZE]
    y_train = y_train[:SAMPLE_SIZE]
    X_test = X_test[:SAMPLE_SIZE]
    y_test = y_test[:SAMPLE_SIZE]

y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [11]:
rnn_type = ['RNN', 'GRU', 'LSTM']
resoult = []
epochs = 100
#clf = discriminator.BasicModel().build(input_shape=(5,5), bidirectional=False, vat=True)
clf = discriminator._RNN(bidirectional=True, input_shape=X_train.shape)
resoult.append(clf.train(X_train, X_test, y_train, y_test, batch_size=128, epochs=epochs, vat=False, early_stop=False))
#score = clf.evaluate(X_test, y_test)#, verbose=0)
score = clf.model.evaluate(X_test, y_test, verbose=0)
print("finish: use_dropout, use_vat: score=%s, accuracy=%s" % (score[0], score[1]))


#clf = discriminator._GRU(bidirectional=True)
#resoult.append(clf.train(X_train, X_test, y_train, y_test, epochs=epochs))
#clf = discriminator._LSTM(bidirectional=True)
#resoult.append(clf.train(X_train, X_test, y_train, y_test, epochs=epochs))


Train on 500 samples, validate on 500 samples
Epoch 1/100
500/500 [==============================] - 2s 3ms/step - loss: 0.5970 - acc: 0.7040 - val_loss: 0.2467 - val_acc: 0.9340
Epoch 2/100
500/500 [==============================] - 0s 883us/step - loss: 0.2881 - acc: 0.9080 - val_loss: 0.2157 - val_acc: 0.9320
Epoch 3/100
500/500 [==============================] - 0s 885us/step - loss: 0.2229 - acc: 0.9040 - val_loss: 0.1926 - val_acc: 0.9680
Epoch 4/100
500/500 [==============================] - 0s 901us/step - loss: 0.1837 - acc: 0.9540 - val_loss: 0.1222 - val_acc: 0.9840
Epoch 5/100
500/500 [==============================] - 0s 923us/step - loss: 0.1248 - acc: 0.9800 - val_loss: 0.0776 - val_acc: 0.9880
Epoch 6/100
500/500 [==============================] - 0s 885us/step - loss: 0.1013 - acc: 0.9880 - val_loss: 0.0719 - val_acc: 0.9880
Epoch 7/100
500/500 [==============================] - ETA: 0s - loss: 0.0852 - acc: 0.984 - 0s 917us/step - loss: 0.0804 - acc: 0.9880 - val_loss

Epoch 61/100
500/500 [==============================] - 0s 950us/step - loss: 0.0050 - acc: 0.9960 - val_loss: 0.0876 - val_acc: 0.9900
Epoch 62/100
500/500 [==============================] - 0s 966us/step - loss: 0.0110 - acc: 0.9940 - val_loss: 0.0917 - val_acc: 0.9900
Epoch 63/100
500/500 [==============================] - 0s 965us/step - loss: 0.0093 - acc: 0.9940 - val_loss: 0.0843 - val_acc: 0.9900
Epoch 64/100
500/500 [==============================] - 0s 962us/step - loss: 0.0087 - acc: 0.9940 - val_loss: 0.0875 - val_acc: 0.9900
Epoch 65/100
500/500 [==============================] - 0s 955us/step - loss: 0.0046 - acc: 0.9960 - val_loss: 0.0905 - val_acc: 0.9900
Epoch 66/100
500/500 [==============================] - 0s 958us/step - loss: 0.0050 - acc: 0.9960 - val_loss: 0.0902 - val_acc: 0.9900
Epoch 67/100
500/500 [==============================] - 0s 956us/step - loss: 0.0074 - acc: 0.9940 - val_loss: 0.0928 - val_acc: 0.9900
Epoch 68/100
500/500 [==========================

In [12]:
sco = clf.model.predict(X_conv_test)
print (sco)

[[8.7404716e-01 7.7260789e-03]
 [1.0000000e+00 8.7488115e-01]
 [8.7685621e-01 7.3902220e-03]
 [8.7755805e-01 7.1403109e-03]
 [9.9999976e-01 2.4541345e-04]
 [8.7404716e-01 7.7260789e-03]
 [9.9999976e-01 2.4431085e-05]
 [8.2742184e-01 1.0068606e-02]
 [9.9999964e-01 3.0794505e-05]
 [8.7816602e-01 6.8925093e-03]
 [8.7823617e-01 6.6529890e-03]
 [9.9999845e-01 2.5108275e-05]
 [8.7874347e-01 6.6418773e-03]
 [8.7978780e-01 6.1603123e-03]
 [9.9993825e-01 4.2869739e-05]
 [9.9706775e-01 1.3854957e-04]
 [1.0000000e+00 6.8502635e-01]
 [1.0000000e+00 9.5464068e-04]
 [1.0000000e+00 8.6682709e-03]
 [1.0000000e+00 6.9544383e-04]
 [1.0000000e+00 6.2199861e-06]
 [8.8140583e-01 5.6928098e-03]
 [9.9999809e-01 2.2416833e-05]
 [8.7940490e-01 6.3914089e-03]
 [8.8315380e-01 5.2668266e-03]
 [9.9999857e-01 2.0485171e-05]
 [8.3551913e-01 7.0027807e-03]
 [9.9972266e-01 8.4292282e-05]
 [8.6400473e-01 8.0170743e-03]
 [1.0000000e+00 2.1734675e-05]
 [9.9999583e-01 8.7531606e-07]
 [8.1372595e-01 1.0506130e-02]
 [1.0000

In [ ]:
#x = range(epochs)
#for i in range(len(resoult)):
#    plt.plot(x, resoult[i].history['acc'], label=opt[i])
#plt.title("binary train accuracy")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.show()
#for i in range(len(resoult)):
#    plt.plot(x, resoult[i].history['val_acc'], label=opt[i])
#plt.title("binary test accuracy")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.show()
#for i in range(len(resoult)):
#    plt.plot(x, resoult[i].history['loss'], label=opt[i])
#plt.title("binary train loss")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.ylim([0,1])
#plt.show()
#for i in range(len(resoult)):
#    plt.plot(x, resoult[i].history['val_loss'], label=opt[i])
#plt.title("binary test loss")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.ylim([0,1])
#plt.show()